# Build the Neural Network
Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

https://pytorch.org/docs/stable/nn.html

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available. Let’s check to see if torch.cuda or torch.backends.mps are available, otherwise we use the CPU.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
# We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__.
# Every nn.Module subclass implements the operations on input data in the forward method.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
# We create an instance of NeuralNetwork, and move it to the device, and print its structure.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


In [10]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0143, -0.2359,  0.0525, -0.0214,  0.4823,  0.2760,  0.1851,  0.2280,
          0.1425, -0.0992,  0.3416,  0.0462,  0.6495,  0.5614, -0.2581, -0.1326,
          0.0017, -0.0579,  0.2432,  0.0779],
        [-0.1947, -0.0913,  0.0550, -0.1470,  0.1089,  0.1861, -0.2263, -0.0049,
          0.1185,  0.3399, -0.0331,  0.1278,  0.5208,  0.7294, -0.1233, -0.0622,
         -0.1389, -0.0708,  0.2300,  0.0874],
        [-0.3029, -0.4159,  0.3804, -0.0134,  0.3444,  0.3588, -0.0566, -0.1722,
          0.1317, -0.1892,  0.0775,  0.0936,  0.5782,  0.6789, -0.2446, -0.2956,
         -0.3691,  0.1763, -0.1797, -0.0143]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0143, 0.0000, 0.0525, 0.0000, 0.4823, 0.2760, 0.1851, 0.2280, 0.1425,
         0.0000, 0.3416, 0.0462, 0.6495, 0.5614, 0.0000, 0.0000, 0.0017, 0.0000,
         0.2432, 0.0779],
        [0.0000, 0.0000, 0.0550, 0.0000, 0.1089, 0.1861, 0.0000, 0.0000, 0.1185,
         0.3399, 0.0000, 0.1278, 0.5208, 0.7294, 0.00

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [16]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0353,  0.0302, -0.0036,  ..., -0.0353, -0.0071,  0.0280],
        [ 0.0072, -0.0201,  0.0233,  ...,  0.0237, -0.0261, -0.0103]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0133, -0.0068], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0129, -0.0094, -0.0078,  ..., -0.0091, -0.0283,  0.0254],
        [ 0.0154,  0.0124,  0.0376,  ...,  0.0135, -0.0342,  0.0360]],
       device='cuda:0', grad_fn=<Sl